# A*-search using `aima-python`

Algorithm implementations taken from [here](https://github.com/aimacode/aima-python/blob/master/search-4e.ipynb)

* *State* is defined by gifts in bags

* *Goal states* are defined by filled bags satisfying problem conditions

* *Actions* : put a gift in a bag with a minimal weight

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [20]:
import numpy as np

In [21]:
from search import Problem, astar_search, uniform_cost_search, state_sequence, action_sequence
import sys
sys.path.append('../common')
from utils import weight2 as weight_fn

In [31]:
n_bags = 25
max_weight = 50


n_horses = 1000
n_balls = 1100
n_bikes = 500
n_trains = 1000
n_coals = 166
n_books = 1200
n_dolls = 1000
n_blocks = 1000
n_gloves = 200

available_gifts = {
    "horse": n_horses,
    "ball": n_balls,
    "bike": n_bikes,
    "train": n_trains,
    "coal": n_coals,
    "book": n_books,
    "doll": n_dolls,
    "blocks": n_blocks,
    "gloves": n_gloves
}

type_cost = {
    "horse": 2,
    "ball": 3,
    "bike": 2,
    "train": 2,
    "coal": 1,
    "book": 5,
    "doll": 2,
    "blocks": 1,
    "gloves": 5    
}

gift_types = available_gifts.keys()
n_types = 8

State is tuple (bags) of tuples (gifts)

In [32]:
def bag_weight(bag, count=1):
    weight = []
    for c in range(count):
        w = 0
        for gift in bag:
            gift_type = gift.split('_')[0]
            w += weight_fn(gift_type, 500)
        weight.append(w)            
    return np.mean(weight)

In [33]:
class SantasBagsProblem(Problem):
            
    def actions(self, state):
        """Return a list of actions executable in this state."""                        
        # find a bag with a minimal weight                
        min_weight_bag_index = 0
        min_weight = self.max_weight
        for i, bag in enumerate(state):
            w = self.bag_weight_fn(bag)
            if min_weight > w:
                min_weight_bag_index = i
                min_weight = w
        
        actions = [(min_weight_bag_index, gift_type) for gift_type in self.gift_types]
#         print("actions: ", actions)
        return actions
    
    def result(self, state, action):
        """The state that results from executing this action in this state."""
        bag_id, gift_type  = action
        if self.available_gifts[gift_type] < 1:
#             print("No more gifts of type : ", gift_type)
            return state
        
#         print("-- result : input state: ", state, "action: ", action)
        bag_weight = self.bag_weight_fn(state[bag_id])
        
        gift_weight = self.weight_fn(gift_type, 500)
        if bag_weight + gift_weight > self.max_weight:
            return state
                
        new_state = list(state)
        gift = gift_type + '_%i' % self.available_gifts[gift_type]
        self.available_gifts[gift_type] -= 1
        bag = list(new_state[bag_id])
        bag.append(gift)
        new_state[bag_id] = tuple(bag)
        
#         print("-- result : output state: ", new_state)
        return tuple(new_state)

    def is_goal(self, state):
        """True if the state is a goal."""        
        mean_score = self._validation(state)
        if mean_score > self.goal_score:
            print("Mean score : ", mean_score)
        return mean_score > self.goal_score

    def step_cost(self, state, action, result=None):
        """The cost of taking this action from this state."""
        bag_id, gift_type  = action        
        return self.type_cost[gift_type]  # Override this if actions have different costs
                
    def _validation(self, state, count=5):
        scores = np.zeros(count)
        for c in range(count):
            score = 0
            for bag in state:
                total_weight_ = self.bag_weight_fn(bag)
                if total_weight_ < self.max_weight:
                    score += total_weight_
            scores[c] = score
        return np.mean(scores)

In [34]:
alpha = 0.80
goal_score = n_bags*max_weight*alpha
print("Goal score: ", goal_score)

Goal score:  1000.0


In [35]:
initial_state=tuple([()]*n_bags)
p = SantasBagsProblem(initial=initial_state,
                      gift_types=list(gift_types)[:n_types], 
                      available_gifts=available_gifts.copy(), 
                      max_weight=max_weight,    
                      type_cost=type_cost,
                      weight_fn=weight_fn,
                      bag_weight_fn=bag_weight,
                      goal_score=goal_score)

Define heuristic function :


In [54]:
def h1(state):
    h = 0
    for bag in state:
        w = bag_weight(bag)
        h += max(max_weight*alpha/n_bags - w, 0.0)
    return h

def h2(state):     
    h = 0
    for bag in state:
        h += bag_weight(bag)
    h = max(goal_score - h, 0.0)
    return h

def h3(state):
    h = 0
    for bag in state:
        w = bag_weight(bag)
        h += max(max_weight/n_bags - w, 0.0)
    return h

def h4(state):     
    h = 0
    for bag in state:
        h += bag_weight(bag)
    h = max(max_weight*n_bags - h, 0.0) / n_bags
    return h


def final_heuristic_fn(state):    
    return np.max(np.array([h1(state), h2(state), h3(state), h4(state)]))

In [55]:
h1(initial_state), h2(initial_state), h3(initial_state), h4(initial_state)

(40.000000000000014, 1000.0, 50.0, 50.0)

In [56]:
result = astar_search(p, final_heuristic_fn)
print(result)

Mean score :  1009.22734646
<Node (('coal_122', 'coal_92'), ('coal_121',), ('coal_120', 'coal_79'), ('coal_119',), ('coal_118', 'coal_89'), ('coal_117', 'coal_80'), ('coal_116', 'coal_90'), ('coal_115', 'coal_91'), ('coal_114',), ('coal_113', 'coal_82'), ('coal_112', 'bike_428'), ('coal_111',), ('coal_110', 'bike_418'), ('coal_109',), ('coal_108',), ('coal_107', 'coal_93'), ('bike_440', 'bike_429'), ('coal_105', 'coal_87'), ('coal_104', 'coal_81'), ('bike_437', 'bike_431'), ('coal_102', 'coal_85'), ('coal_101', 'coal_86'), ('coal_100', 'bike_422'), ('bike_433', 'coal_96'), ('coal_98', 'coal_83')): 52>


In [57]:
result.state

(('coal_122', 'coal_92'),
 ('coal_121',),
 ('coal_120', 'coal_79'),
 ('coal_119',),
 ('coal_118', 'coal_89'),
 ('coal_117', 'coal_80'),
 ('coal_116', 'coal_90'),
 ('coal_115', 'coal_91'),
 ('coal_114',),
 ('coal_113', 'coal_82'),
 ('coal_112', 'bike_428'),
 ('coal_111',),
 ('coal_110', 'bike_418'),
 ('coal_109',),
 ('coal_108',),
 ('coal_107', 'coal_93'),
 ('bike_440', 'bike_429'),
 ('coal_105', 'coal_87'),
 ('coal_104', 'coal_81'),
 ('bike_437', 'bike_431'),
 ('coal_102', 'coal_85'),
 ('coal_101', 'coal_86'),
 ('coal_100', 'bike_422'),
 ('bike_433', 'coal_96'),
 ('coal_98', 'coal_83'))

In [58]:
h1(result.state), h2(result.state), h3(result.state), h4(result.state)

(0.0, 0.0, 0.0, 9.6881155879204783)

In [59]:
p._validation(result.state)

1006.2655133864812

In [ ]:
submission_file = '../results/submission_' + \
                  str(datetime.now().strftime("%Y-%m-%d-%H-%M")) + \
                  '.csv'

In [ ]:
def write_submission(state, filename):
    with open(filename, 'w') as w:
        w.write("Gifts\n")
        for bag in state:
            w.write(' '.join(bag) + '\n')
    
write_submission(result.state, submission_file)